# Installing and importing packages

In [ ]:
! pip install tweepy

In [ ]:
! pip install PyDrive

In [ ]:
# Run this cell to import the packages you will need to unpack the dataset
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import pickle
import os
import zipfile
import scipy.ndimage
from google.colab import files

import tweepy
import sys
import json
import codecs
from tweepy.parsers import JSONParser

# Getting new dataset that is cleaned

In [ ]:
### authentication -- run once 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:

### lists the files in the folder 
file_list = drive.ListFile({'q': "'18sX8Dymv_AAEaY7mJxbmh_LjljD8oRwK' in parents and trashed=false"}).GetList() ## long string in the url of folder
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: true_ans_df, id: 1iSOAZSYZErO9Fq5w1_qSalMP-x8DaVPKO1-SKz1eVOA
title: wrong_ans_df.csv, id: 1_j6ilNKGL71BqM262Vg8KTTiu9jT6ete
title: true_ans_df.csv, id: 1drwwnkULVoMfKRJBkj-mCyaIlbRtPli5
title: data_VEC, id: 1re1jFEQvnyeGuryQkl_dWaNoLnLBkN8Y44cIa6LqeJg
title: data_VEC.csv, id: 1_oB0pdMdaHbVl6YllHKDFJGukhhfK-Iz
title: Data CLEAN, id: 1pgellPyb7ynUHU64Prm0xZwI4-sqfw3M
title: Original Tweets, id: 1yDewPt327TxTrKCvuCIpHNyigDMaKPyv
title: bully_victim1.csv, id: 1ngNQ_sLWIAnN7scAxJl4-y8QZB_soeTQ
title: bully_victim.csv, id: 1qud97IBO3rkLkmDJTi-Y8U--waH7v6tb
title: data_categorical_multi.csv, id: 10qfXUcLUW4GPBM1ra3_cRBpnW9ltQEh-
title: data_categorical_binary.csv, id: 1Mn0cT-De4mq-9gJ7J2s201jvZQ1bWNGK
title: glove.6B.100d.txt, id: 1-FOMH_1xZcjtYo1uW3_2BwozqHFC-dZb
title: glove.6B.zip, id: 1tuFKpS8F1lhnaemAYi_w498Vli6WGuKP
title: data_tweet_clean_500_stop.csv, id: 1bUddQJ0LejpEROhpWmMtYYSoXvR2q4pc
title: data_Tweet_clean.csv, id: 1FNa4-dHR6aEeunRIwMnaBBacSlCHZOsW
title: Medication_1, i

In [ ]:
### download the file 
file_downloaded = drive.CreateFile({'id': '1Mn0cT-De4mq-9gJ7J2s201jvZQ1bWNGK'})
file_downloaded.GetContentFile('data_categorical_binary.csv')

df_data = pd.read_csv('data_categorical_binary.csv', dtype = object)
df_data.head(15)

,Unnamed: 0,Tweet ID,User ID,Bullying_Traces?,Type,Form,Teasing?,Author_Role,Emotion,Text
0,0,107688644067856384,185389094,1,1,1,1,1,1,@bellathorne143 i herd that you got bullied wh...
1,1,102206417217392640,226320672,0,0,0,0,0,0,"Bullying: O gesto mais idiota, estpido e irrac..."
2,2,102779484725448704,297557032,0,0,0,0,0,0,The Bully at School Goes High Tech Part 1: \n...
3,3,108676972149874688,157724561,0,0,0,0,0,0,Esse @Felipemath fazendo bullying comigo! Haha'
4,4,106590106873372672,62179998,0,0,0,0,0,0,AUISHUAHS eu e o @wallace_mancha tiramos o dia...
5,5,109034091743154176,177913822,0,0,0,0,0,0,"@Looweihao @haoyangg @sleepybed Hello, since w..."
6,6,102533497637437441,70412906,1,1,1,1,1,3,For those keeping score at home- cousin who ju...
7,7,103679008855691264,11363462,0,0,0,0,0,0,"BETTER ANTI-BULLYING AD SLOGANS: Hey, Bullies...."
8,8,105202476927549441,293593896,0,0,0,0,0,0,@luttylutz bahh lo kaga demen dia juga? Gue ki...
9,9,107278521805713408,150964152,0,0,0,0,0,0,cara3: bully iam gara2 uda di ucpin tpi gk bls'


In [ ]:


#print(df_data["Text"][0:].replace('b'', ''))
                                  
print(df_data["Text"].replace("b\'", ''))



0       @bellathorne143 i herd that you got bullied wh...
1       Bullying: O gesto mais idiota, estpido e irrac...
2       The Bully at School Goes High Tech  Part 1: \n...
3        Esse @Felipemath  fazendo bullying comigo! Haha'
4       AUISHUAHS eu e o @wallace_mancha tiramos o dia...
5       @Looweihao @haoyangg @sleepybed Hello, since w...
6       For those keeping score at home- cousin who ju...
7       BETTER ANTI-BULLYING AD SLOGANS: Hey, Bullies....
8       @luttylutz bahh lo kaga demen dia juga? Gue ki...
9         cara3: bully iam gara2 uda di ucpin tpi gk bls'
10      http://t.co/nbBZDxf Please help to stop workpl...
11      @lucy_robbo yes on thur, she said she might, w...
12      @bbuk @brianofficial bobby needs threw out tha...
13      don't you get it? This is me getting rid of yo...
14      No  bullying se eu jogar um tijolo na sua cara...
15      @BillyBlancO_ @DjLBully nigga used to be a rea...
16      theres going to be bullys and ignorant people ...
17      @_Maca

# Authenticating Twitter Process

In [ ]:
####### Access Information #################

# Parameter you need to specify
consumer_key = '7w9qPKMQzbCNvx4cjVoaLqjjt'
consumer_secret = 'bj8lNT1yJTkZHlIbg6n7NEBTX73oQttmbPy7nCUuEcYEuldt7O'
access_key = '4439149653-6eK3Z25ODtI3sg7rv3NqNLAWmO6vz270Lewq1tD'
access_secret = 'AJtrhAdfUmWjWnirvASSsp5PEjDbiNwO5XBDxeSuLypxs'

inputFile = 'tweet_id'
outputFile = 'tweet.json'

#############################################

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth_handler=auth, parser=JSONParser())

# Just testing out text extraction and adding it as a feature

In [ ]:
#extracting the text sample (EXAMPLE)

# print(tweet.keys())
# for val in tweet.keys():
#   print(val, tweet[val])
#text = tweet['text'].encode('ascii', 'ignore')

In [ ]:

#text

In [ ]:
### concatenating a list of zeros to dataframe
# l = pd.Series(list(np.zeros(df_data.shape[0])))
# df_data_sample = pd.concat([df_data, l], axis = 1)
# df_data_sample.head(5)
# for val in df_data['Text']:
#   listEdit = list(val)
#   listEdit.pop(0)
#   listEdit.pop(0)
#   str1 = ''.join(listEdit)
#   #print(str1)
# print(df_data.head(25))

In [ ]:
# for i in range(0,df_data.shape[0]):
# #for val in df_data['Text']:
#   listEdit = list(df_data['Text'][i])
#   listEdit.pop(0)
#   listEdit.pop(0)
#   str1 = ''.join(listEdit)
#   #print(str1)
#   df_data['Text'][i] = str1
#df_data.head(10)
print(df_data['Text'])

0       @bellathorne143 i herd that you got bullied wh...
1       Bullying: O gesto mais idiota, estpido e irrac...
2       The Bully at School Goes High Tech  Part 1: \n...
3        Esse @Felipemath  fazendo bullying comigo! Haha'
4       AUISHUAHS eu e o @wallace_mancha tiramos o dia...
5       @Looweihao @haoyangg @sleepybed Hello, since w...
6       For those keeping score at home- cousin who ju...
7       BETTER ANTI-BULLYING AD SLOGANS: Hey, Bullies....
8       @luttylutz bahh lo kaga demen dia juga? Gue ki...
9         cara3: bully iam gara2 uda di ucpin tpi gk bls'
10      http://t.co/nbBZDxf Please help to stop workpl...
11      @lucy_robbo yes on thur, she said she might, w...
12      @bbuk @brianofficial bobby needs threw out tha...
13      don't you get it? This is me getting rid of yo...
14      No  bullying se eu jogar um tijolo na sua cara...
15      @BillyBlancO_ @DjLBully nigga used to be a rea...
16      theres going to be bullys and ignorant people ...
17      @_Maca

In [ ]:

from google.colab import files
#files.download('data_Tweet_clean.csv')

# Still experimenting

In [ ]:
! pip install nltk

In [ ]:

text_feature = list()

In [ ]:
indices_to_drop = list()
for i in range(0, 500):
  try:
    tweet = api.get_status(int(df_data['Tweet ID'][i]))
    text = tweet['text'].encode('ascii', 'ignore')
    text_feature.append(text)   
  except tweepy.error.TweepError:
    print(tweepy.error.TweepError)
    indices_to_drop.append(i)
    pass

  if i % 50 == 0:
    print(i)


0
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
50
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
100
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
150
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
200
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
250
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
300
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
350
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
400
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.

In [ ]:
len(indices_to_drop)

33

In [ ]:

api.wait_on_rate_limit

False

In [ ]:
from nltk.corpus import stopwords
from nltk import *
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
# s0 = "This is a cooool #dummysmiley: :-) :-P <3 and some arrows < > -> <--"
# tknzr.tokenize(s0)
# #from nltk.book import *
# stopList = stopwords.words('spanish')
# print(stopList)
#tomato = FreqDist(list(text1))
# keyList = tomato.keys()
# newList = []
# for word in keyList:
#     if word.isalpha():
#         if word not in stopList:
#             newList.append(word)
# listTomato2 = sorted(newList, key = lambda x:tomato[x],reverse=True)
# print(listTomato2[:50])
import nltk
nltk.download('stopwords')

english_stop_list = stopwords.words('english')
spanish_stop_list = stopwords.words('spanish')
french_stop_list = stopwords.words('french')
for i in range(0,df_data.shape[0]):
  text_list = tknzr.tokenize(df_data['Text'][i])
  newList = []
  for word in text_list:
     if ((word not in english_stop_list) and (word not in spanish_stop_list) and (word not in french_stop_list)):
      newList.append(word)
  print(newList)
  str1 = ' '.join(newList)
  print(str1)
  df_data['Text'][i] = str1

In [ ]:
df_data.to_csv('data_tweet_clean_500_stop.csv')

from google.colab import files
files.download('data_tweet_clean_500_stop.csv')

In [ ]:
for i in range(0,500):
  tweet = api.get_status(int(df_data['Tweet ID'][i]))
  print(tweet['user']['age'])

AttributeError: ignored

In [ ]:
stopList = stopwords.words("english")
tomato = FreqDist(list(newList))
keyList = tomato.keys()
newList = []
for word in keyList:
   if word.isalpha():
       if word not in stopList:
           newList.append(word)
listTomato2 = sorted(newList, key = lambda x:tomato[x],reverse=True)
print(listTomato2[:50])


['How', 'I', 'serious', 'question', 'many', 'children', 'victims', 'bullying', 'deal']
